# Random Forest

---
## Experimental Setup

### Libraries & Settings

In [1]:
import os              # General OS commands
import numpy as np     # NumPy
import pandas as pd    # Python Data Analysis Library
import zipfile         # Compress/decompress ZIP files
import sqlite3         # SQLite3 Database Driver
import re              # Regular Expressions

In [2]:
from sklearn import ensemble, metrics

In [3]:
# Never truncate columns, display all the data
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', -1)

# Display floating-point numbers with 4 decimals in `pandas.DataFrame`
pd.options.display.float_format = '{:,.4f}'.format

import matplotlib.pyplot as plt
# Display MatPlotLib stuff inline
%matplotlib inline

### Database

In [4]:
zip_filename = "../../data/ee-insee-2005_custom-sqlite.zip"
eedb = zip_filename.replace("-sqlite.zip", ".sqlite")

if not os.path.exists(eedb):
    with zipfile.ZipFile(zip_filename) as zip_file:
        zip_file.extractall("../../data/")

In [5]:
with sqlite3.connect(eedb) as con:
    query = "SELECT * FROM eec05_custom"
    eec05 = pd.read_sql_query(query, con)

In [6]:
# # Create a ("female" x "enfant") interaction variable
# bool_ = eec05[["enfants_", "female_"]].astype(bool)
# eec05["female_enfants_"] = (bool_.enfants_ & bool_.female_).astype(int)

# # Drop data we don't need
# eec05 = eec05[eec05.age60_ == 0]
# eec05 = eec05.drop("age60_", 1)

---
## Random Forest

In [7]:
# Build a list of parameters to include in the model, using regex
# https://www.datarobot.com/blog/multiple-regression-using-statsmodels/
filters = {
#     "trim": "^trim$",
    "age": "^age$",
    "female": "^female_$",
    
#     "nationalite": "^nat28_[0-9]*_$",
#     "matri": "^matri_[0-9]*_$",
#     "enfants": "^nbenf18_[0-9]*_$",
    
#     "diploma": "^dip_[0-9]*_$",
#     "specialite": "^spe_[0-9]*_$",
    
#     "region": "^reg_[0-9]*_$",
    "unite_urbaine": "^tur5_[0-9]*_",
    
#     "csp_pere": "^cspp_[0-9]*_$",
#     "csp": "^cstot_[0-9]*_$"
}

params = {k: sorted([x for x in eec05.columns if re.match(r, x)]) for (k, r) in filters.items()}

# Avoid the dummy variable trap
params = {k: (v if len(v) == 1 else v[:-1]) for (k, v) in params.items()}
# params["region"] += ["region2_"]

In [8]:
trims = ["t{}".format(x) for x in sorted(eec05["trim"].unique())]
eec05_ = {t: eec05[eec05.trim == int(t[1])] for t in trims}
X = {t: eec05_[t][sum(params.values(), [])] for t in eec05_}
y = {t: eec05_[t]["actop_"] for t in eec05_}

### Train the model using `trim=1` data

In [9]:
# Create & fit the forest
random_forest_sk = ensemble.RandomForestClassifier(
    criterion='gini',
    n_estimators=100,
    max_features=4,
    max_depth=12,
    random_state=1,
    n_jobs=-1
).fit(X["t1"], y["t1"])

In [10]:
# Check the accuracy of the model on the training set
print "Accuracy: {}".format(random_forest_sk.score(X["t1"], y["t1"]))

Accuracy: 0.822493036212


### Apply the model to test data (`trim=2/3/4`)

In [11]:
# Check the accuracy of the model on the training & test sets

for t in trims:
#     predicted = random_forest_sk.predict_proba(X[t]).T[1]
#     predicted = np.where(predicted < 0.5, 0, 1)
    predicted = random_forest_sk.predict(X[t])
    print "Accuracy ({}): {}".format(t, metrics.accuracy_score(y[t], predicted))

    predicted_proba = random_forest_sk.predict_proba(X[t]).T[1]
    print "ROC Area-Under-Curve ({}): {}".format(t, metrics.roc_auc_score(y[t], predicted_proba))

# Note: the two methods for computing `predicted` are equivalent (THRESHOLD = 0.5)

Accuracy (t1): 0.822493036212
ROC Area-Under-Curve (t1): 0.891215112582
Accuracy (t2): 0.82119822172
ROC Area-Under-Curve (t2): 0.889983633811
Accuracy (t3): 0.816366535496
ROC Area-Under-Curve (t3): 0.885738897542
Accuracy (t4): 0.816052548184
ROC Area-Under-Curve (t4): 0.885801370369


---
## Marginal Effects

In [12]:
marginal_effects = pd.DataFrame()

### By modifying the dataset ("passage de tout le monde en licence")

In [13]:
brute_force = pd.Series()
for category in filters:
    for pivot in params[category]:
        non_pivots = [x for x in params[category] if x != pivot]
        
        X_one = X["t1"].copy() 
        X_one[non_pivots] = 0
        X_one[pivot] = 1
        
        proba_one = random_forest_sk.predict_proba(X_one).T[1].mean()
        proba_t1 = random_forest_sk.predict_proba(X["t1"]).T[1].mean()
        brute_force[pivot] = proba_one - proba_t1

In [14]:
marginal_effects["brute_force"] = brute_force
marginal_effects.transpose()

,age,tur5_1_,tur5_2_,tur5_3_,tur5_4_,female_
brute_force,0.4814,-0.0323,-0.0045,0.0166,0.0151,0.0417


---
## Most important parameters
_ranked by **entropy importance**_ (**TODO**: where did Bastien get this?!)

In [15]:
entropy = pd.DataFrame(random_forest_sk.feature_importances_, index=X["t1"].columns, columns=["entropy"])
entropy.sort_values(by="entropy", ascending=False).transpose()

,age,female_,tur5_1_,tur5_2_,tur5_4_,tur5_3_
entropy,0.9674,0.0215,0.0043,0.0024,0.0024,0.0021
